In [23]:
import matplotlib.pyplot as plt
from collections import defaultdict
import re

In [ ]:
import rasterio

In [ ]:
import numpy as np

In [ ]:
import os

In [ ]:
from osgeo import gdal

# Input directory and filenames
image_dir = 'images'
filenames = [
    '3DIMG_18JUN2024_0000_L1C_SGP_V01R00_IMG_VIS.tif',
    '3DIMG_18JUN2024_0000_L1C_SGP_V01R00_IMG_MIR.tif',
    '3DIMG_18JUN2024_0000_L1C_SGP_V01R00_IMG_SWIR.tif',
    '3DIMG_18JUN2024_0000_L1C_SGP_V01R00_IMG_WV.tif',
    '3DIMG_18JUN2024_0000_L1C_SGP_V01R00_IMG_TIR1.tif',
    '3DIMG_18JUN2024_0000_L1C_SGP_V01R00_IMG_TIR2.tif',
]

input_paths = [os.path.join(image_dir, f) for f in filenames]

# Create VRT in memory
vrt_options = gdal.BuildVRTOptions(separate=True)
vrt = gdal.BuildVRT('stacked.vrt', input_paths, options=vrt_options)

# Translate VRT to GeoTIFF
gdal.Translate(
    'stacked.tif',
    vrt,
    creationOptions=[
        'COMPRESS=DEFLATE',
        'TILED=YES'
    ]
)

print("✅ Stacked GeoTIFF created: stacked.tif")


In [ ]:
# Open output file for update
ds = gdal.Open('stacked.tif', gdal.GA_Update)

# Set descriptions per band
band_labels = [
    'VIS', 'MIR', 'SWIR', 'WV', 'TIR1', 'TIR2'
]


for i, label in enumerate(band_labels, start=1):
    band = ds.GetRasterBand(i)
    band.SetDescription(label)

ds.FlushCache()
ds = None


In [ ]:
import rasterio

# Open the multiband image
with rasterio.open('stacked.tif') as src:
    print(f"Number of bands: {src.count}")
    print(f"Width: {src.width}, Height: {src.height}")
    print(f"CRS: {src.crs}")
    print(f"Bounds: {src.bounds}")
    print(src.descriptions)

    # Read metadata of each band
    for i in range(1, src.count + 1):
        band = src.read(i)
        print(f"Band {i} - dtype: {band.dtype}, shape: {band.shape}")

In [ ]:
def normalize(array):
    array_min, array_max = array.min(), array.max()
    return (array - array_min) / (array_max - array_min)

In [ ]:
import rasterio

with rasterio.open('stacked.tif') as src:
    band = src.read(1)
    # band = normalize(band)

plt.imshow(band, cmap='gray')
plt.title('Band 1')
plt.colorbar()
plt.show()

In [ ]:
!gdalinfo stacked.tif

In [ ]:
from glob import glob

# Set directory containing the .tif files
input_dir = "images/INSAT3DR-L1C"
output_dir = "./stacked"
os.makedirs(output_dir, exist_ok=True)

# Define band stacking order
band_order = ['VIS', 'SWIR', 'MIR', 'TIR1', 'TIR2', 'WV']

# Group files by timestamp
pattern = re.compile(r"3DIMG_(\d{2}[A-Z]{3}\d{4})_(\d{4})_.*_IMG_(\w+)\.tif")
grouped_files = defaultdict(dict)

for filepath in glob(os.path.join(input_dir, "*.tif")):
    match = pattern.search(os.path.basename(filepath))
    if match:
        date, time, band = match.groups()
        timestamp = f"{date}_{time}"
        grouped_files[timestamp][band] = filepath

# Stack and save
for timestamp, files_dict in grouped_files.items():
    if all(band in files_dict for band in band_order):
        stack = []
        meta = None
        for band in band_order:
            with rasterio.open(files_dict[band]) as src:
                data = src.read(1)
                stack.append(data)
                if meta is None:
                    meta = src.meta.copy()
        stack_array = np.stack(stack)
        meta.update(count=len(band_order))
        output_path = os.path.join(output_dir, f"{timestamp}_stack.tif")
        with rasterio.open(output_path, "w", **meta) as dst:
            dst.write(stack_array)
        print(f"Saved: {output_path}")
    else:
        print(f"Skipping {timestamp}: incomplete band set")



In [ ]:
import os
from glob import glob
import rasterio
import rasterio.errors

input_dir = "images/INSAT3DR-L1C"
unreadable_files = []

for filepath in glob(os.path.join(input_dir, "*.tif")):
    try:
        with rasterio.open(filepath) as src:
            pass
    except rasterio.errors.RasterioIOError:
        unreadable_files.append(filepath)

if unreadable_files:
    unreadable_files.sort()
    print("Unreadable files:")
    for f in unreadable_files:
        print(f)
else:
    print("All files are readable.")



In [ ]:
import numpy as np
import rasterio

with rasterio.open("stacked/18JUN2024_0030_stack.tif") as src:
    img = src.read()  # shape: (C, H, W)
    print("Shape:", img.shape)


In [ ]:
!gdalinfo "stacked/18JUN2024_0630_stack.tif"

In [ ]:
import os

files = sorted([f for f in os.listdir('resized/') if f.endswith('_stack.tif')])
stacked_list = []

for f in files:
    path = os.path.join('resized/', f)
    with rasterio.open(path) as src:
        arr = src.read()
        stacked_list.append(src)

In [ ]:
directory = 'resized/'
pattern = '_stack.tif'

In [ ]:
from pathlib import Path
directory = Path(directory)
files = sorted([f for f in directory.iterdir() 
               if f.is_file() and f.name.endswith(pattern)])
if not files:
    print(f"No files found matching pattern '{pattern}' in {directory}")

stacked_arrays = []
for file_path in files:
    try:
        with rasterio.open(file_path) as src:
            # Read all bands
            arr = src.read()
            stacked_arrays.append(arr)
            print(f"Loaded: {file_path.name} - Shape: {arr.shape}")
    except Exception as e:
        print(f"Error loading {file_path}: {e}")
        continue
print(f"Successfully loaded {len(stacked_arrays)} raster files")

In [ ]:
stacked_arrays = np.array(stacked_arrays)

In [ ]:
input_len = 4
pred_len = 1

In [29]:
T = stacked_arrays.shape[0]
N = T - input_len - pred_len + 1
X, y = [], []
for i in range(N):
    X.append(stacked_arrays[i : i + input_len])
    y.append(stacked_arrays[i + input_len : i + input_len + pred_len])


In [30]:
sequence = stacked_arrays

In [31]:
def display_grid(stacked_list, file_names, band=0, figsize=(15, 10)):
    """Display all images in a grid layout."""
    n_images = len(stacked_list)
    cols = min(4, n_images)  # Max 4 columns
    rows = (n_images + cols - 1) // cols
    
    fig, axes = plt.subplots(rows, cols, figsize=figsize)
    if n_images == 1:
        axes = [axes]
    elif rows == 1:
        axes = [axes]
    else:
        axes = axes.flatten()
    
    for i, (arr, name) in enumerate(zip(stacked_list, file_names)):
        # Handle multi-band images - display specified band
        if arr.ndim == 3:
            img_data = arr[band]  # Select band
        else:
            img_data = arr
            
        im = axes[i].imshow(img_data, cmap='viridis')
        axes[i].set_title(f'{name}\nBand {band}')
        axes[i].axis('off')
        plt.colorbar(im, ax=axes[i], shrink=0.8)
    
    # Hide empty subplots
    for i in range(n_images, len(axes)):
        axes[i].axis('off')
    
    plt.tight_layout()
    plt.show()

In [27]:
import os
file_names = [f for f in os.listdir('resized/') if f.endswith(pattern)]

file_names.sort()
file_names

['18JUN2024_0000_stack.tif',
 '18JUN2024_0030_stack.tif',
 '18JUN2024_0100_stack.tif',
 '18JUN2024_0130_stack.tif',
 '18JUN2024_0200_stack.tif',
 '18JUN2024_0230_stack.tif',
 '18JUN2024_0300_stack.tif',
 '18JUN2024_0330_stack.tif',
 '18JUN2024_0400_stack.tif',
 '18JUN2024_0530_stack.tif',
 '18JUN2024_0600_stack.tif',
 '18JUN2024_0630_stack.tif',
 '18JUN2024_0700_stack.tif',
 '18JUN2024_0730_stack.tif',
 '18JUN2024_0800_stack.tif',
 '18JUN2024_0830_stack.tif',
 '18JUN2024_1030_stack.tif',
 '18JUN2024_1100_stack.tif',
 '18JUN2024_1130_stack.tif',
 '18JUN2024_1200_stack.tif',
 '18JUN2024_1230_stack.tif',
 '18JUN2024_1300_stack.tif',
 '18JUN2024_1330_stack.tif',
 '18JUN2024_1400_stack.tif',
 '18JUN2024_1430_stack.tif',
 '18JUN2024_1500_stack.tif',
 '18JUN2024_1530_stack.tif',
 '18JUN2024_1600_stack.tif']

In [ ]:
display_grid(stacked_arrays, file_names)

TypeError: list indices must be integers or slices, not tuple

In [ ]:
import rasterio 

with rasterio.open('resized/18JUN2024_0030_stack.tif') as src:
    print(src.width, src.height, src.width / src.height)


In [ ]:
scale = 0.25

In [ ]:
import os
import glob
import rasterio
from rasterio.transform import Affine
from rasterio.enums import Resampling

input_folder = "stacked/"
output_folder = "resized/"
scale_factor = 0.25

for file in glob.glob(os.path.join(input_folder, "*.tif")):
    with rasterio.open(file) as src:
        width = int(src.width * scale_factor)
        height = int(src.height * scale_factor)

        new_res_x = (src.bounds.right - src.bounds.left) / width
        new_res_y = (src.bounds.top - src.bounds.bottom) / height
        new_transform = Affine.translation(src.bounds.left, src.bounds.top) * Affine.scale(new_res_x, -new_res_y)

        kwargs = src.meta.copy()
        kwargs.update({
            'transform': new_transform,
            'width': width,
            'height': height
        })

        dst_path = os.path.join(output_folder, os.path.basename(file))

        with rasterio.open(dst_path, 'w', **kwargs) as dst:
            for i in range(1, src.count + 1):
                src_data = src.read(
                    i,
                    out_shape=(height, width),
                    resampling=Resampling.bilinear
                )
                dst.write(src_data, i)


In [ ]:
import rasterio.plot


with rasterio.open('stacked/18JUN2024_0330_stack.tif') as src:
    img = src.read(1)
    rasterio.plot.show(img)

In [ ]:
import numpy as np

stacked_array = np.stack([rasterio.open(os.path.join('stacked', f)).read() for f in files])